<a href="https://colab.research.google.com/github/martaarozarena/KSchool-Master-Final-Project/blob/master/exog_varaibles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
#from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/KSchool-Master-Final-Project'       # path to your project on Drive

drive.mount(ROOT)           # we mount the drive at /content/drive

#PROJECT_PATH = join(ROOT, PROJ)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/.shortcut-targets-by-id/1JF1t24jwhTtQKP-lLGdAUHcNKaXnHy3u/KSchool-Master-Final-Project/data

/content/drive/.shortcut-targets-by-id/1JF1t24jwhTtQKP-lLGdAUHcNKaXnHy3u/KSchool-Master-Final-Project/data


In [3]:
import pandas as pd

In [ ]:
sheets=["c1_schoolclosing","c2_workplaceclosing","c3_cancelpublicevents","c4_restrictionsongatherings","c5_closepublictransport","c6_stayathomerequirements","c7_domestictravel","c8_internationaltravel"]
url = "https://drive.google.com/file/d/1P37xzRtsjWIMUfhs3uUrHHLyToyE7qOJ/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
variables = []
for i in sheets:
  varc = pd.read_excel(path,sheet_name=sheets)


In [ ]:
j=1
for i in varc.keys():
  varc[i]=varc[i].drop("CountryCode",axis=1)
  varc[i].index=varc[i].CountryName
  varc[i]=varc[i].drop("CountryName",axis=1)
  varc[i]=varc[i][:-3]
  varc[i]=varc[i].T
  varc[i].index=pd.to_datetime(varc[i].index,format="%d%b%Y")
  varc[i]=varc[i].truncate("01-01-2020","30-06-2020")
  varc[i].to_csv('variable_c{}.csv'.format(j))
  j=j+1

In [ ]:
varc1=varc['c1_schoolclosing']
varc2=varc["c2_workplaceclosing"]
varc3=varc['c3_cancelpublicevents']
varc4=varc["c4_restrictionsongatherings"]
varc5=varc['c5_closepublictransport']
varc6=varc["c6_stayathomerequirements"]
varc7=varc['c7_domestictravel']
varc8=varc["c8_internationaltravel"]

In [7]:
url = "https://drive.google.com/file/d/1P37xzRtsjWIMUfhs3uUrHHLyToyE7qOJ/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
sheets2=["h1_publicinfocampaign","h2_testingpolicy","h3_contacttracing"]
for i in sheets2:
  varh = pd.read_excel(path,sheet_name=sheets2)

In [8]:
k=1
for i in varh.keys():
  varh[i]=varh[i].drop("CountryCode",axis=1)
  varh[i].index=varh[i].CountryName
  varh[i]=varh[i].drop("CountryName",axis=1)
  varh[i]=varh[i][:-3]
  varh[i]=varh[i].T
  varh[i].index=pd.to_datetime(varh[i].index,format="%d%b%Y")
  varh[i]=varh[i].truncate("01-01-2020","30-06-2020")
  varh[i].to_csv('variable_h{}.csv'.format(k))
  k=k+1

In [ ]:
#download the yougov data and then choosing the mask wearing data (i12 variable)

In [58]:
var_percent_mask = pd.read_csv("https://raw.githubusercontent.com/YouGov-Data/covid-19-tracker/master/data/spain.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (78) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
var_percent_mask.columns

Index(['RecordNo', 'endtime', 'qweek', 'i1_health', 'i2_health', 'i7a_health',
       'i3_health', 'i4_health', 'i5_health_1', 'i5_health_2',
       ...
       'm14_5', 'm14_6', 'm14_7', 'm14_8', 'm14_9', 'm14_10', 'm14_11',
       'm14_96', 'm14_99', 'm14_open'],
      dtype='object', length=206)

In [61]:
var_percent_mask=var_percent_mask.loc[:,["endtime","i12_health_1","qweek"]]


In [62]:
var_percent_mask["endtime"] = pd.to_datetime(var_percent_mask["endtime"])

In [63]:
var_percent_mask.index=var_percent_mask["endtime"]

In [64]:
var_percent_mask.drop("endtime",axis=1)

,i12_health_1,qweek
endtime,,
2020-02-04 08:13:00,Always,week 1
2020-02-04 08:16:00,Not at all,week 1
2020-02-04 08:16:00,Not at all,week 1
2020-02-04 08:16:00,Always,week 1
2020-02-04 08:17:00,Always,week 1
...,...,...
2020-07-26 06:35:00,Always,week 12
2020-07-26 06:41:00,Always,week 12
2020-07-26 06:41:00,Always,week 12


In [71]:
#replace strings for numeric values 1-always, 2-frequently 3-sometimes 4-rarely 5-not at all
var_percent_mask.i12_health_1.replace(["Always","Frequently","Sometimes","Rarely","Not at all"], [1,2, 3, 4, 5],inplace=True)


In [85]:
var_percent_mask.groupby("qweek").count()

,endtime,i12_health_1
qweek,,
week 1,1005,1005
week 10,999,999
week 11,1032,1032
week 12,1007,1007
week 2,1010,1010
week 3,1005,1005
week 4,986,986
week 5,1004,1004
week 6,1002,1002
